In [1]:
import pickle as pkl
import trimesh

# download the SMPL code from https://smpl.is.tue.mpg.de/
# put it to denoisfm/data_generation/smpl_webuser
from denoisfm.data_generation.smpl_webuser.serialization import load_model

scene = trimesh.Scene()

# download the SMAL model from https://smal.is.tue.mpg.de/
# put it to denoisfm/data_generation/smal_online
model_path = 'denoisfm/data_generation/smal_online/smal_CVPR2017.pkl'
model = load_model(model_path)

### Template

In [4]:
import denoisfm.utils.preprocessing_util as preprocessing_util
import denoisfm.utils.fmap_util as fmap_util
import denoisfm.utils.remesh_util as remesh_util
import torch
import numpy as np

# Template is the mean model
# model.r are the mesh vertices, and model.f are the mesh faces.
verts_orig = torch.tensor(model.r, dtype=torch.float32)
faces_orig = torch.tensor(model.f.astype(np.int64), dtype=torch.long)

# normalize the face area to 1
verts_orig = preprocessing_util.normalize_face_area(verts_orig, faces_orig)
   
# remesh to break SMAL connectivity   
verts_r, faces_r = remesh_util.remesh_simplify_iso(
    verts=verts_orig,
    faces=faces_orig,
    simplify_strength=1, # no simplification, keep 100% of the faces
    )

# save the remeshed template
template_remeshed = trimesh.Trimesh(
    vertices=verts_r,
    faces=faces_r
    )
template_remeshed.export('data/template/animal/template.off')

# correspondences to SMAL shapes
corr_r = fmap_util.nn_query(
    verts_r,
    verts_orig, 
    )
np.savetxt(
    'data/template/animal/corr.txt',
    corr_r.numpy() + 1, fmt='%d'
    )

In [ ]:
# view the remeshed template
scene.geometry.clear()
scene.add_geometry(trimesh.creation.axis(origin_size=0.1))

scene.add_geometry(template_remeshed)

scene.set_camera()
scene.show()

### Generate other poses for each train shape

In [3]:
# base directory where you store the data
base_dir = '/lustre/mlnvme/data/s94zalek_hpc-shape_matching/data_denoisfm'

In [4]:
train_names = f'{base_dir}/test/SMAL_r/train.txt'

# read the train names, e.g. lion_01, horse_02
with open(train_names) as f:
    train_names = f.readlines()
    train_names = [x.strip() for x in train_names]
    
print('Names of the training shapes:', train_names)

Names of the training shapes: ['lion_01', 'horse_02', 'horse_03', 'cougar_01', 'cougar_02', 'cow_01', 'cow_02', 'cow_05', 'dog_03', 'dog_02', 'fox_03', 'fox_04', 'horse_05', 'hippo_05', 'hippo_02', 'hippo_03', 'hippo_04', 'lion_03', 'lion_04', 'horse_06', 'horse_07', 'horse_09', 'horse_10', 'cow_07', 'dog_04', 'dog_06', 'dog_07', 'cougar_03', 'cougar_04', 'dog_09', 'cow_08', 'wolf_02']


In [5]:
# read name map from train names to the names of fitted parameters
# e.g. 00028740_ferrari horse_01
name_map = f'{base_dir}/test/SMAL_r/name_map.txt'

with open(name_map) as f:
    name_map = f.readlines()
    name_map = [x.strip() for x in name_map]
    
    name_map = dict([x.split()[::-1] for x in name_map])
    
print('Name map:', name_map)

Name map: {'horse_01': '00028740_ferrari', 'horse_02': '00047093_ferrari', 'horse_03': '00049424_ferrari', 'horse_04': '00057894_ferrari', 'cougar_01': '00211799_ferrari', 'lion_01': '450-122410176-lions-natural-habitat', 'dog_01': 'Brown-And-White-Akita-Dog_alph', 'cougar_02': 'cougar', 'cow_01': 'cow_alph', 'cow_02': 'cow_alph4', 'cow_03': 'cow_alph5', 'cow_04': 'cow2', 'cow_05': 'Davis_cow_00000', 'dog_02': 'dog_alph', 'dog_03': 'dog2', 'fox_01': 'fox', 'fox_02': 'fox_alph', 'fox_03': 'fox-05', 'fox_04': 'fox-06', 'horse_05': 'grazing', 'hippo_01': 'hippo_alpha', 'hippo_02': 'hippo_alpha_mouthopen2', 'hippo_03': 'Hippo_for_Nat', 'hippo_04': 'hippo_walking', 'hippo_05': 'hippo5', 'hippo_06': 'hippos', 'lion_02': 'lion_yawn', 'lion_03': 'lion3', 'lion_04': 'lion6', 'lion_05': 'MaleLion800', 'horse_06': 'muybridge_014_01', 'horse_07': 'muybridge_030_02', 'horse_08': 'muybridge_071_04', 'horse_09': 'muybridge_074_01', 'horse_10': 'muybridge_075_04', 'cow_06': 'muybridge_076_04', 'cow_07

## Generate 64k training animals

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import os


shape_list = []

iterator = tqdm(train_names)
for train_name in iterator:
    
    iterator.set_description(train_name)

    orig_name = name_map[train_name]

    orig_path = f'{base_dir}/test/SMAL_fitted/{orig_name}.pkl'
    with open(orig_path, 'rb') as orig_path:
        shape_data = pkl.load(orig_path, encoding="latin1")
        
        
    for i in range(6000):
        
        # randomize the shape parameter
        model.betas[:] = np.zeros_like(model.betas)
        model.betas[:20] = np.array(shape_data['beta']) + 0.05 * np.random.randn(*shape_data['beta'].shape)

        # randomize the pose parameter
        model.pose[:] = np.array(shape_data['pose']) + 0.2 * np.random.randn(*shape_data['pose'].shape)
        model.pose[0:3]=0

        model.trans[:] = 0.

        shape_list.append(torch.tensor(model.r.copy(), dtype=torch.float32))
       
shape_list = torch.stack(shape_list)

save_dir = f'{base_dir}/train/base/SMAL_{len(shape_list)}'
os.makedirs(save_dir, exist_ok=True)

# save the vertices
torch.save(
    shape_list, 
    f'{save_dir}/verts.pt'
)
# save the faces
torch.save(
    faces_orig, 
    f'{save_dir}/faces.pt'
)
 

In [ ]:
import torch
import numpy as np

# read 10 random meshes from the training set and plot them
scene.geometry.clear()

random_indices = np.random.choice(shape_list.shape[0], 10, replace=False)

for i, idx in enumerate(random_indices):
    mesh = trimesh.Trimesh(
        vertices=shape_list[idx].numpy(),
        faces=model.f
        )
    
    mesh.vertices += np.array([2*i, 0, 0])
    
    scene.add_geometry(mesh)
    
scene.set_camera()
scene.show()